In [58]:
import os
import json
import pandas as pd
import traceback

In [59]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

In [60]:
load_dotenv()
KEY=os.getenv('OPENAI_API_KEY')

In [61]:
llm=ChatOpenAI(openai_api_key=KEY,model_name='gpt-3.5-turbo', temperature=0.5)

Imports:

In [62]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [63]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
      "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
      "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}

In [64]:
TEMPLATE="""
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guid. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [65]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE
)

In [66]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [67]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a multiple Choice quiz for {subject} student.\
You need to evaluate the complexity of the question and given a complete analysis of the quiz. Only use at max 50 words for complexity.
if the quiz is not at per with the cognitive and analytical of the students,\
update the quiz questions which needs to be changed and change the tone such that is perfectly fits the student abilities.
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [68]:
quiz_evaluation_prompt=PromptTemplate(input_variables=['subject', 'quiz'], template=TEMPLATE)

In [69]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt,output_key='review', verbose=True)

In [70]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [71]:
file_path=r'/Users/mendye/Development/github/mcqgen/data.txt'

In [72]:
with open(file_path, 'r') as file:
        TEXT=file.read()

In [73]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [74]:
NUMBER=5
SUBJECT="Machine Learning"
TONE='SIMPLE'

In [75]:
with get_openai_callback() as cb: 
        response=generate_evaluate_chain(
                {
                        "text": TEXT,
                        "number": NUMBER,
                        "subject": SUBJECT,
                        "tone": TONE,
                        "response_json": json.dumps(RESPONSE_JSON)
                }
        )

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: See also: Timeline of machine learning
The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work 

In [76]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 2525
Prompt Tokens: 1802
Completion Tokens: 723
Total Cost: 0.004149


In [81]:
quiz=response.get('quiz')
quiz=json.loads(quiz)

In [85]:
quiz_table_data = []

for key, value in quiz.items():
    mcq = value['mcq']
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value['options'].items()
        ]
    )
    correct = value['correct']
    quiz_table_data.append({'MCQ': mcq, "Choices": options, 'Correct': correct})

In [86]:
quiz_table_data

[{'MCQ': 'Who coined the term machine learning in 1959?',
  'Choices': 'a: Donald Hebb | b: Tom M. Mitchell | c: Arthur Samuel | d: Walter Pitts',
  'Correct': 'c'},
 {'MCQ': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'Choices': 'a: Cybertron | b: HAL 9000 | c: Skynet | d: R2-D2',
  'Correct': 'a'},
 {'MCQ': 'Who proposed the early mathematical models of neural networks in the study of human cognitive systems?',
  'Choices': 'a: Donald Hebb | b: Tom M. Mitchell | c: Warren McCulloch | d: Walter Pitts',
  'Correct': 'c'},
 {'MCQ': 'According to Tom M. Mitchell, what does a computer program do when it learns from experience with respect to some class of tasks and performance measure?',
  'Choices': 'a: It makes coffee | b: It improves its performance | c: It plays chess | d: It shuts down',
  'Correct': 'b'},
 {'MCQ': 'What are the two objectives of modern-day machine learning?',
  'Choices': 'a: To cook dinner and do l

In [88]:
quiz=pd.DataFrame(quiz_table_data)

In [92]:
quiz.to_csv("machineLearning.csv", index=False)